In [ ]:
!python -m pip install --upgrade pip

!curl https://pyenv.run | bash

text = r"""
# Load pyenv automatically by appending
# the following to
# ~/.bash_profile if it exists, otherwise ~/.profile (for login shells)
# and ~/.bashrc (for interactive shells) :

export PYENV_ROOT="$HOME/.pyenv"
[[ -d $PYENV_ROOT/bin ]] && export PATH="$PYENV_ROOT/bin:$PATH"
eval "$(pyenv init -)"

# Restart your shell for the changes to take effect.

# Load pyenv-virtualenv automatically by adding
# the following to ~/.bashrc:

eval "$(pyenv virtualenv-init -)"
"""
with open("~/.bashrc", "a") as f:
    f.write(text)    


!pip install poetry 

!mkdir ~/mlx_week_7
os.chdir("~/mlx_week_7")


In [ ]:
!apt-get update && apt-get -y install vim rsync coreutils git-lfs less htop


In [ ]:
python -m pip install --upgrade pip
! pip install tensorflow accelerate peft bitsandbytes trandformer
!pip install matplotlib timm albumentations transformers datasets

In [ ]:
# Conda cheat sheet
# https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

In [1]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
!pip install accelerate bitsandbytes
!pip install -U transformers


In [ ]:
quantization_config = BitsAndBytesConfig()

In [ ]:
!pip install -U accelerate bitsandbytes

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

: 

: 

: 

NameError: name 'model' is not defined

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["w1", "w2", "w3"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)



In [5]:

CUTOFF_LEN = 256
tokenizer.pad_token = "!"


dataset = load_dataset("harpreetsahota/modern-to-shakesperean-translation")
print("dataset", dataset)
train_data = dataset["train"]


def generate_prompt(user_query):
    sys_msg = "Translate the given text to Shakespearean style."
    p = (
        "<s> [INST]"
        + sys_msg
        + "\n"
        + user_query["modern"]
        + "[/INST]"
        + user_query["shakespearean"]
        + "</s>"
    )
    return p


tokenize = lambda prompt: tokenizer(
    prompt + tokenizer.eos_token,
    truncation=True,
    max_length=CUTOFF_LEN,
    padding="max_length",
)
train_data_modified = train_data.shuffle().map(
    lambda x: tokenize(generate_prompt(x)), remove_columns=["modern", "shakespearean"]
)

dataset DatasetDict({
    train: Dataset({
        features: ['modern', 'shakespearean'],
        num_rows: 274
    })
})


Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [11]:
train_data[0:1]

{'modern': ["When someone says 'She's thirsty, ain't she?', they're implying she's seeking attention."],
 'shakespearean': ['When one remarks, "She doth crave attention, doth she not?", they suggest her desire for notice.']}

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data_modified,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=6,
        learning_rate=1e-4,
        logging_steps=2,
        optim="adamw_torch",
        save_strategy="epoch",
        output_dir="mixtral-moe-lora-instruct-shapeskeare",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


model.config.use_cache = False
trainer.train()